# How to use the final dataset : YOLO, RT-DETR

### Merge all nine WC folders and create train-test-val split

In [ ]:
import os
import shutil
from img_bbox import train_test_val_folders

PATH="FINAL_SUBIMAGES/"
os.makedirs(PATH + "all/images", exist_ok=True)
os.makedirs(PATH + "all/labels", exist_ok=True)
folder_path = PATH + "all/"

# Iterate through each folder and move files to the new subfolders
for folders in os.listdir(PATH):
    if folders != ".DS_Store" and folders != "all":
        folder_name = PATH + folders + "/"
        for subfolder in ["images/", "labels/"]:
            for filename in os.listdir(folder_name + subfolder):
                if filename.endswith(".jpg") or filename.endswith(".txt"):
                    shutil.move(folder_name + subfolder + filename, folder_path + subfolder + filename)

train_test_val_folders(folder_path, 0.7, 0.15)
os.rmdir(folder_path+"images")
os.rmdir(folder_path+"labels") # delete former non-empty folders

### Write the corresponding .yaml to train Ultralytics model

In [ ]:
yaml_path = os.path.join(folder_path, "merged_subimages.yaml")

with open(yaml_path, 'w') as yaml_file:
    yaml_file.write(    # the train,val and test paths should be absolute
f"""train: {folder_path}train/images 
val: {folder_path}val/images
test: {folder_path}test/images
nc: 1
names: ['orange']
"""
    )
print(f"YAML file created at: {yaml_path}")

#### train YOLO architecture

It usually takes several hours to train with these hyperparameters.  
If you are running the code on a server via an SSH connection, create a .py file and paste the code from the cell below.  
Then, run the following command in the terminal (with the environment activated) to keep the code running even after disconnecting from the server: :  
nohup python your_file.py > nohup_outputs/your_file.out &

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov10x.pt")                                     # 3 GPUs
model.train(data=yaml_path, epochs=300, imgsz=640, dropout=0.2, batch=((12*3)), patience=20, save_period=20, device=(0,1,2), name="train_10x")

#### validate the model

In [ ]:
model.val(data=yaml_path, batch=((12*3)), imgsz=640, device=(0,1,2), name="val_10x") # the .yaml will automatically assign the validation set

model.predict(source="image_path_or_url", save=True, imgsz=640, conf=0.5) # Used to run the model on new, unseen data to generate predictions. 
# It does not provide detailed performance metrics but allows you to see how the model performs on real-world images.

#### train RT-DETR

all previous remarks for YOLO also work here

In [ ]:
from ultralytics import RTDETR

model = RTDETR("rtdetr-l.pt")
model.train(data=yaml_path, epochs=300, imgsz=640, dropout=0.2, batch=((12*3)), patience=20, save_period=20, device=(0,1,2), name="train_rtdetr_l")